
# 🧠 Simple Business Intelligence Agent with LangGraph

This notebook builds a LangGraph-based AI agent that:
- Analyzes sales, cost, and customer data
- Calculates profit, CAC, and trends
- Uses OpenAI GPT to generate business recommendations

---

> ⚠️ **Note:** This notebook uses OpenAI's GPT model. You need to set your OpenAI API key to run it:
```python
import os
os.environ["OPENAI_API_KEY"] = "sk-..."  # Replace with your key
```

Due to billing limitations, this notebook has not been executed, but all logic and structure is complete and test-ready.


In [ ]:
!pip install -U langgraph langchain openai langchain-openai


In [ ]:
import os

if "OPENAI_API_KEY" not in os.environ:
    os.environ["OPENAI_API_KEY"] = input("Enter your OpenAI API key: ")


In [5]:
from langchain_openai import ChatOpenAI
from langgraph.graph import StateGraph, END
from typing import TypedDict, List, Optional
from datetime import datetime


In [6]:
# 📊 Define state for the agent
class BusinessData(TypedDict):
    today_revenue: float
    today_cost: float
    today_customers: int
    yesterday_revenue: float
    yesterday_cost: float
    yesterday_customers: int

class AgentState(TypedDict, total=False):
    input_data: BusinessData
    profit: float
    cac_today: float
    cac_change_percent: float
    revenue_change_percent: float
    cost_change_percent: float
    alerts: List[str]
    recommendations: Optional[str]


In [7]:
# 🔁 Input Node
def input_node(state: AgentState) -> AgentState:
    return state


In [8]:
# 🧠 Processing Node
def processing_node(state: AgentState) -> AgentState:
    data = state['input_data']
    profit = data['today_revenue'] - data['today_cost']

    cac_today = data['today_cost'] / data['today_customers'] if data['today_customers'] else 0
    cac_yesterday = data['yesterday_cost'] / data['yesterday_customers'] if data['yesterday_customers'] else 0

    cac_change = ((cac_today - cac_yesterday) / cac_yesterday) * 100 if cac_yesterday else 0
    revenue_change = ((data['today_revenue'] - data['yesterday_revenue']) / data['yesterday_revenue']) * 100 if data['yesterday_revenue'] else 0
    cost_change = ((data['today_cost'] - data['yesterday_cost']) / data['yesterday_cost']) * 100 if data['yesterday_cost'] else 0

    alerts = []
    if profit < 0:
        alerts.append("⚠️ Negative profit today.")
    if cac_change > 20:
        alerts.append("⚠️ CAC increased by more than 20%.")

    return {
        **state,
        "profit": profit,
        "cac_today": cac_today,
        "cac_change_percent": cac_change,
        "revenue_change_percent": revenue_change,
        "cost_change_percent": cost_change,
        "alerts": alerts,
    }


In [9]:
# 💬 Recommendation Node (uses OpenAI)
llm = ChatOpenAI(model="gpt-3.5-turbo", temperature=0.3)

def recommendation_node(state: AgentState) -> AgentState:
    summary = f"""
Today’s profit is {state['profit']:.2f}.
Revenue changed by {state['revenue_change_percent']:.2f}%.
Cost changed by {state['cost_change_percent']:.2f}%.
CAC is {state['cac_today']:.2f} ({state['cac_change_percent']:.2f}% change).
Alerts: {', '.join(state['alerts']) if state['alerts'] else 'None'}.
"""
    prompt = f"""
Given the following business summary, provide concise, actionable recommendations:

{summary}

Answer in 2-3 short bullet points.
"""

    result = llm.invoke(prompt)
    return {
        **state,
        "recommendations": result.content.strip()
    }


In [10]:
# 🔧 Build the LangGraph
builder = StateGraph(AgentState)
builder.add_node("input", input_node)
builder.add_node("process", processing_node)
builder.add_node("recommend", recommendation_node)

builder.set_entry_point("input")
builder.add_edge("input", "process")
builder.add_edge("process", "recommend")
builder.add_edge("recommend", END)

graph = builder.compile()


In [ ]:
# 🚀 Run the agent with sample input
input_data = {
    "input_data": {
        "today_revenue": 1500,
        "today_cost": 1000,
        "today_customers": 50,
        "yesterday_revenue": 1000,
        "yesterday_cost": 700,
        "yesterday_customers": 50,
    }
}

result = graph.invoke(input_data)

import json
print(json.dumps(result, indent=2))


In [ ]:
# ✅ Simple test
def test_agent():
    out = graph.invoke({
        "input_data": {
            "today_revenue": 500,
            "today_cost": 800,
            "today_customers": 20,
            "yesterday_revenue": 1000,
            "yesterday_cost": 600,
            "yesterday_customers": 30,
        }
    })

    assert "profit" in out
    assert "recommendations" in out
    assert isinstance(out["recommendations"], str)
    assert out["profit"] == -300
    print("✅ Test passed!")

test_agent()



## 🧾 Sample Expected Output

```json
{
  "profit": 500.0,
  "cac_today": 20.0,
  "cac_change_percent": 25.0,
  "revenue_change_percent": 50.0,
  "cost_change_percent": 42.86,
  "alerts": [
    "⚠️ CAC increased by more than 20%"
  ],
  "recommendations": "- Monitor CAC trends.\n- Reallocate budget to high-performing ads."
}
```
